This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [11]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [12]:
# DATASET DIRECTORY CONFIGURATION
train_dir = "train-cat-rabbit" 
val_dir = "val-cat-rabbit"
test_dir = "test-images"

In [4]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [13]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    # validation_split=0.2 <--- REMOVED this line as data is now pre-split
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    # subset='training' <--- REMOVED
)
# val_generator now flows from the separate validation directory
val_generator = test_datagen.flow_from_directory(
    val_dir,  # Flows from the dedicated 'val-cat-rabbit' folder
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False # Typically set to False for validation data
    # subset='validation' <--- REMOVED
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 1600 images belonging to 2 classes.
Found 414 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


In [21]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting and L2 regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', 
                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.3), # CHANGED from 0.5 to 0.3
    layers.Dense(1, activation='sigmoid')
])

In [23]:
# Configure the model optimizers, loss function, and metrics
# Define the Early Stopping callback:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stop] # ADDED the callback
)

Epoch 1/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 193ms/step - accuracy: 0.5544 - loss: 1.1002 - val_accuracy: 0.5121 - val_loss: 0.9152
Epoch 2/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 177ms/step - accuracy: 0.6062 - loss: 0.8711 - val_accuracy: 0.6449 - val_loss: 0.8161
Epoch 3/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 176ms/step - accuracy: 0.7212 - loss: 0.7046 - val_accuracy: 0.6667 - val_loss: 0.7198
Epoch 4/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 13s 258ms/step - accuracy: 0.7588 - loss: 0.6326 - val_accuracy: 0.7077 - val_loss: 0.6344
Epoch 5/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 318ms/step - accuracy: 0.7788 - loss: 0.5945 - val_accuracy: 0.6594 - val_loss: 0.6722
Epoch 6/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 314ms/step - accuracy: 0.7819 - loss: 0.5594 - val_accuracy: 0.6763 - val_loss: 0.6267
Epoch 7/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 18s 351ms/step - accuracy: 0.7719 - loss: 0.5579 - val_accuracy: 0.7440 - val_loss: 0.5834
Epoch 8/20
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 312ms/step - accuracy: 0.7925 - loss: 0.5288 - val_accura

In [25]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8667 - loss: 0.4604
Test Accuracy: 0.8666666746139526


In [26]:
# SAVE THE MODEL
model.save('exercise_6_custom_Cabales&Juanico.h5')

In [34]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model_path='exercise_6_custom_Cabales&Juanico.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # prob_class_0 is P(Cat) based on flow_from_directory's alphabetical order
    prob_class_0 = model.predict(img_array)[0,0]
    
    # Determine the predicted class and its confidence
    if prob_class_0 >= 0.5:
        label = "Rabbit" # Swapped
        confidence = prob_class_0
    else:
        label = "Cat" # Swapped
        # P(Cat) = 1 - P(Rabbit)
        confidence = 1.0 - prob_class_0
        
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

In [37]:
# Example usage:
predict_image("test-images/cat/cat.210.jpg")
predict_image("test-images/rabbit/rabbit_test.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


Prediction: Cat (confidence: 0.95)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Prediction: Rabbit (confidence: 1.00)
